In [0]:
from pyspark.sql.functions import col, current_timestamp, lit
from datetime import datetime

record_date = datetime.today().strftime('%Y-%m-%d')

# 1. Read both landing tables
df_tracks = spark.read.table("lastfmdata.bronze.landing_tracks")
df_artists = spark.read.table("lastfmdata.bronze.landing_artists")

# 2. Filter tracks and convert duration to INT
df_tracks_clean = df_tracks \
    .filter((col("duration").isNotNull()) & (col("duration") != "0")) \
    .withColumn("duration_in_seconds", col("duration").cast("int"))

# 3. Join with artist table
df_joined = df_tracks_clean.alias("t") \
    .join(
        df_artists.alias("a"),
        (col("t.artist_name") == col("a.name")) & 
        (col("t.country") == col("a.country")) & 
        (col("t.record_date") == col("a.record_date")),
        how="left"
    )

# 4. Select required columns
df_online = df_joined.select(
    col("t.artist_name"),
    col("t.name").alias("track_name"),
    col("duration_in_seconds"),
    col("t.url").alias("track_url"),
    col("a.url").alias("artist_url"),
    col("t.country"),
    col("t.record_date")
).dropDuplicates(["record_date", "artist_name", "track_name"]) \
 .withColumn("load_timestamp", current_timestamp())

#  # Ensure the record_date matches the specified record_date
# df_online_filtered = df_online.filter(col("record_date") == record_date)

# 5. Write to silver table
df_online.write \
  .format("delta") \
  .mode("overwrite")  \
  .saveAsTable("lastfmdata.silver.online_artist_tracks")
